In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torchvision import models
import torch
import gym, sys
import cv2
from tqdm import tqdm
from IPython.display import clear_output
from array2gif import write_gif
from rlxai.dp.img import data_transform


/opt/conda/envs/xai/lib/python3.8/site-packages/ale_py/roms/__init__.py:94: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  _RESOLVED_ROMS = _resolve_roms()


In [2]:
env = gym.make("Breakout-v4") # Deterministic
random_seed = 215161  # set random seed if required (0 = no random seed)
action_dim = env.action_space.n
device = "cuda:1"
from rlxai.agent.agent_selection import get_agent
model_name = "PER"
save_path = "./../model"
model,_,_ = get_agent(model_name=model_name,save_path=save_path,device=device,action_dim=action_dim)

A.L.E: Arcade Learning Environment (version +978d2ce)
[Powered by Stella]


load
...Load model from ../model/PER/model.pt...


In [3]:
if random_seed:
    print("--------------------------------------------------------------------------------------------")
    print("setting random seed to ", random_seed)
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)

--------------------------------------------------------------------------------------------
setting random seed to  215161


In [4]:
ACTION_MEANING = env.unwrapped.get_action_meanings()

In [5]:
img_collection = []
state = env.reset()
state, r, done, info = env.step(1)
state_list = [data_transform(state) for _ in range(3)]
img_collection.append(state.transpose((2, 0, 1)))
iter = 0
ACTION_LIST = []
R = 0
last_lives = info['lives']
while True:
    state = np.concatenate(state_list ,axis=1) / 255.0
    action_dict = model.select_action(state,False)
    if info["lives"] < last_lives :
        action = 1
        last_lives = info["lives"]
    else :
        action = int(action_dict["action"])
    s, r, done, info = env.step(action)
    R += r
    ACTION_LIST.append(ACTION_MEANING[action])
    #print(ACTION_MEANING[int(action_dict["action"])] , info )
    #############################################
    state_list.pop(0)
    cur_lives = info["lives"]
    if last_lives > cur_lives :
        s = np.zeros_like(s)
    state_list.append(data_transform(s))
    #############################################
    img_collection.append(s.transpose((2, 0, 1)))
    iter += 1
    if done:
        break
print(", ".join(ACTION_LIST))
print(f"Reward : {R}")
if R < 10 :
    print(f"Reward : {R} is very small so we don't make gif file")
else :
    file_path = "./breakout.gif"
    write_gif(img_collection, file_path)
    print(f"Reward : {R} save gif...")

FIRE, RIGHT, FIRE, FIRE, RIGHT, FIRE, RIGHT, FIRE, LEFT, RIGHT, RIGHT, FIRE, FIRE, LEFT, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, LEFT, FIRE, FIRE, FIRE, LEFT, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, LEFT, LEFT, FIRE, LEFT, NOOP, NOOP, RIGHT, RIGHT, FIRE, FIRE, LEFT, FIRE, FIRE, LEFT, LEFT, FIRE, RIGHT, FIRE, FIRE, FIRE, RIGHT, RIGHT, FIRE, FIRE, LEFT, LEFT, LEFT, NOOP, FIRE, RIGHT, RIGHT, RIGHT, FIRE, RIGHT, LEFT, LEFT, LEFT, RIGHT, LEFT, RIGHT, LEFT, LEFT, LEFT, NOOP, FIRE, RIGHT, RIGHT, LEFT, LEFT, LEFT, LEFT, LEFT, LEFT, LEFT, LEFT, RIGHT, FIRE, FIRE, RIGHT, FIRE, RIGHT, FIRE, RIGHT, LEFT, FIRE, RIGHT, FIRE, RIGHT, RIGHT, FIRE, RIGHT, FIRE, FIRE, FIRE, RIGHT, RIGHT, FIRE, LEFT, LEFT, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, FIRE, LEFT, FIRE, LEFT, LEFT, LEFT, FIRE, FIRE, RIGHT, FIRE, RIGHT, NOOP, LEFT, RIGHT, LEFT, NOOP, RIGHT, NOOP, FIRE, LEFT, LEFT, FIRE, FIRE, RIGHT, RIGHT, LEFT, FIRE, RIGHT, LEFT, LEFT, FIRE, NOOP, RIGHT, LEFT, LEFT, RIGHT, LEFT, RIG